### Задание

Реализуйте алгоритм классификации метод k ближайших соседей.

Требования к коду:
* Код должен быть хорошо структурирован
* Код должен быть эффективен
* Имплементация должна быть максимально векторизованной и, где это возможно, не использовать циклы

Необходимо реализовать класс KnnBruteClassifier, с реализацией прототипа, представленного ниже.

Должна быть реализована поддержка метрики расстояния L2 (параметр metric) и параметр weights типа 'uniform' и 'distance'.

В качестве входного файла необходимо использовать файл "regr_data_XXX.npy", полученный от бота командой /get seminar04

В качестве решения необходимо отправить боту, указав seminar04 в поле caption,  следующие файлы:
* knn.ipynb - содержит класс, реализующий ваш алгоритм
* knn_results.npy - файл с результатами тестов, который можно будет сгенерировать с помощью этого ноутбука

Для проверки решения после отправки необходимо отправить боту следующую команду:
/check seminar04

В случае возникновения вопросов по интерфейсу смотрите детали реализации класса sklearn.neighbors.KNeighborsClassifier
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

In [149]:
import numpy as np
import numpy.linalg as la

In [150]:
class KnnBruteClassifier(object):
    '''Классификатор реализует взвешенное голосование по ближайшим соседям. 
    Поиск ближайшего соседа осуществляется полным перебором.
    Параметры
    ----------
    n_neighbors : int, optional
        Число ближайших соседей, учитывающихся в голосовании
    weights : str, optional (default = 'uniform')
        веса, используемые в голосовании. Возможные значения:
        - 'uniform' : все веса равны.
        - 'distance' : веса обратно пропорциональны расстоянию до классифицируемого объекта
        -  функция, которая получает на вход массив расстояний и возвращает массив весов
    metric: функция подсчета расстояния (по умолчанию l2).
    '''
    def __init__(self, n_neighbors=1, weights='uniform', metric='l2'):
        self.n_neighbors = n_neighbors
        self.weights = weights
        self.metric = metric
        self.feature = []
        self.target = []
        self.classes = []
     
    def fit(self, x, y):
        '''Обучение модели.
        Парметры
        ----------
        x : двумерным массив признаков размера n_queries x n_features
        y : массив/список правильных меток размера n_queries
        Выход
        -------
        Метод возвращает обученную модель
        '''
        self.feature = x
        self.target = y
        self.classes = set(y)
        return f'KnnBruteClassifier(n_neighbors={self.n_neighbors}, weights={self.weights}, metric={self.metric})'
        
    def predict(self, x):
        """ Предсказание класса для входных объектов
        Параметры
        ----------
        X : двумерным массив признаков размера n_queries x n_features
        Выход
        -------
        y : Массив размера n_queries
        """
        y = np.array([0])
        neigh_dist, neigh_indarray = self.kneighbors(x,self.n_neighbors)
        if self.weights == 'uniform':
          for query_idx in range(len(neigh_indarray)):
            classes_dict = dict.fromkeys(self.classes, 0)
            for query_target in self.target[neigh_indarray[query_idx]]:
              classes_dict[query_target] = classes_dict[query_target] + 1
            y = np.append(y, max(classes_dict, key=classes_dict.get))
        elif self.weights == 'distance':
          for query_idx in range(len(neigh_indarray)):
            classes_dict = dict.fromkeys(self.classes, 0)
            query_target = self.target[neigh_indarray[query_idx]]
            for i in range(len(query_target)):
              classes_dict[query_target[i]] = classes_dict[query_target[i]] + 1/neigh_dist[query_idx][i]
            y = np.append(y, max(classes_dict, key=classes_dict.get))
        else:
          neigh_weight = self.weights(neigh_dist)
          for query_idx in range(len(neigh_indarray)):
            classes_dict = dict.fromkeys(self.classes, 0)
            query_target = self.target[neigh_indarray[query_idx]]
            for i in range(len(query_target)):
              classes_dict[query_target[i]] = classes_dict[query_target[i]] + neigh_weight[query_idx][i]
            y = np.append(y, max(classes_dict, key=classes_dict.get))

        return y[1:]  

        
    def predict_proba(self, X):
        """Возвращает вероятности классов для входных объектов
        Параметры
        ----------
        X : двумерным массив признаков размера n_queries x n_features
        Выход
        -------
        p : массив размера n_queries x n_classes] c вероятностями принадлежности 
        объекта к каждому классу
        """
        p = np.zeros([1, len(self.classes)])
        neigh_dist, neigh_indarray = self.kneighbors(X,self.n_neighbors)
        if self.weights == 'uniform':
          for query_idx in range(len(neigh_indarray)):
            classes_dict = dict.fromkeys(self.classes, 0)
            for query_target in self.target[neigh_indarray[query_idx]]:
              classes_dict[query_target] = classes_dict[query_target] + 1
            prob_dict = {}
            for k, v in classes_dict.items():
                prob_dict[k] = v / sum(classes_dict.values())
            p = np.append(p, [list(prob_dict.values())], axis=0)
        elif self.weights == 'distance':
          for query_idx in range(len(neigh_indarray)):
            classes_dict = dict.fromkeys(self.classes, 0)
            query_target = self.target[neigh_indarray[query_idx]]
            for i in range(len(query_target)):
              classes_dict[query_target[i]] = classes_dict[query_target[i]] + 1/neigh_dist[query_idx][i]
            prob_dict = {}
            for k, v in classes_dict.items():
                prob_dict[k] = v / sum(classes_dict.values())
            p = np.append(p, [list(prob_dict.values())], axis=0)
        else:
          neigh_weight = self.weights(neigh_dist)
          for query_idx in range(len(neigh_indarray)):
            classes_dict = dict.fromkeys(self.classes, 0)
            query_target = self.target[neigh_indarray[query_idx]]
            for i in range(len(query_target)):
              classes_dict[query_target[i]] = classes_dict[query_target[i]] + neigh_weight[query_idx][i]
            prob_dict = {}
            for k, v in classes_dict.items():
                prob_dict[k] = v / sum(classes_dict.values())
            p = np.append(p, [list(prob_dict.values())], axis=0)

        return p[1:]
        
    def kneighbors(self, x, n_neighbors):
        """Возвращает n_neighbors ближайших соседей для всех входных объектов и расстояния до них
        Параметры
        ----------
        X : двумерным массив признаков размера n_queries x n_features
        Выход
        -------
        neigh_dist массив размера n_queries х n_neighbors
        расстояния до ближайших элементов
        neigh_indarray, массив размера n_queries x n_neighbors
        индексы ближайших элементов
        """
        neigh_dist = np.zeros([1,n_neighbors])
        neigh_indarray = np.zeros([1,n_neighbors])
         
        for test_idx, test_query in enumerate(x):
          dist = np.array([0])
          indarray = np.array([0])
          for train_idx, train_x in enumerate(self.feature):
            r = la.norm(test_query-train_x)
            dist = np.append(dist,r)
            indarray = np.append(indarray, train_idx)
          dist = dist[1:]
          indarray = indarray[1:]
          arg_sorted = dist.argsort()
          indarray = indarray[arg_sorted]
          dist = dist[arg_sorted]
          neigh_dist = np.append(neigh_dist, [dist[:n_neighbors]], axis=0)
          neigh_indarray = np.append(neigh_indarray, [indarray[:n_neighbors]], axis=0)
        return neigh_dist[1:], neigh_indarray[1:].astype(int)  

In [151]:
def load_file(filename):
    """
    TODO: Необходимо загрузить файл задания и вернуть словарь с ключами "X_train", "X_test", "y_train"
    """
    with open(filename, 'rb') as f:
        Mydata = np.load(f, allow_pickle=True)
    return Mydata.item()

In [152]:
input_filename = "/content/knn_data_045.npy" #TODO задать путь к входному файлу
data_dict = load_file(input_filename)

In [153]:
model = KnnBruteClassifier(n_neighbors=5, weights='uniform')
model.fit(data_dict["X_train"], data_dict["y_train"])
l2_uniform_n5_y_predict = model.predict(data_dict["X_test"])

In [154]:
model = KnnBruteClassifier(n_neighbors=10, weights='uniform')
model.fit(data_dict["X_train"], data_dict["y_train"])
l2_uniform_10_y_predict = model.predict(data_dict["X_test"])

In [155]:
model = KnnBruteClassifier(n_neighbors=5, weights='distance')
model.fit(data_dict["X_train"], data_dict["y_train"])
l2_distance_n5_y_predict = model.predict(data_dict["X_test"])

In [156]:
output_filename = "knn_results.npy"
result_dict = {
    "input_filename": input_filename,
    "l2_uniform_n5_y_predict": l2_uniform_n5_y_predict,
    "l2_uniform_10_y_predict": l2_uniform_10_y_predict,
    "l2_distance_n5_y_predict": l2_distance_n5_y_predict,
}
np.save(output_filename, result_dict)